In [135]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(device)
block_size = 8
#batch size
batch_size = 4
max_iters = 10000
learning_rate = 3e-4
eval_iters = 250
dropout = 0.2

cuda


In [136]:
with open('the_palace_of_illusions.txt','r',encoding='utf-8') as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocabulary_size = len(chars)

['\n', ' ', '!', '(', ')', '*', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '>', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', '\\', '^', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '}', '£', '«', '¬', '°', '—', '‘', '’', '“', '”']


In [137]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)


In [138]:
#training and validating sets
print("data is ", data) 
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split == 'train' else val_data
    ix = torch.randint(len(data)-block_size,(batch_size,))
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x,y = x.to(device), y.to(device)
    return x,y

x = train_data[:block_size]
y = train_data[1:block_size+1]

for t in range(block_size):
    context = x[:t+1]
    target = y[t]
    print("when input is ",context,' target is ', target)


data is  tensor([43, 58, 68,  ...,  1,  0,  0])
when input is  tensor([43])  target is  tensor(58)
when input is  tensor([43, 58])  target is  tensor(68)
when input is  tensor([43, 58, 68])  target is  tensor(65)
when input is  tensor([43, 58, 68, 65])  target is  tensor(71)
when input is  tensor([43, 58, 68, 65, 71])  target is  tensor(57)
when input is  tensor([43, 58, 68, 65, 71, 57])  target is  tensor(58)
when input is  tensor([43, 58, 68, 65, 71, 57, 58])  target is  tensor(1)
when input is  tensor([43, 58, 68, 65, 71, 57, 58,  1])  target is  tensor(70)


In [139]:
x, y = get_batch('train')
print('inputs: ')
print(x)
print('targets: ')
print(y)

@torch.no_grad()
def estimate_loss():
    out = {}
    model.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_iters)
        for k in range(eval_iters):
            X, Y = get_batch(split)
            logits, loss = model(X, Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    model.train()
    return out

inputs: 
tensor([[55, 70, 70, 55, 68,  1, 70, 58],
        [ 0, 86, 32, 69,  1, 59, 70,  1],
        [70, 71, 51, 70, 55, 54,  1, 51],
        [60, 55, 53, 70, 69,  8,  1, 46]], device='cuda:0')
targets: 
tensor([[70, 70, 55, 68,  1, 70, 58, 51],
        [86, 32, 69,  1, 59, 70,  1, 56],
        [71, 51, 70, 55, 54,  1, 51, 70],
        [55, 53, 70, 69,  8,  1, 46, 58]], device='cuda:0')


In [140]:
class BigramLanguageModel(nn.Module):
     def __init__(self,vocabulary_size):
         super().__init__()
         self.token_embedding_table = nn.Embedding(vocabulary_size,vocabulary_size)
         #learnable model with Embedding table = v * v table
     def forward(self,index,targets=None):
         logits = self.token_embedding_table(index)
         if targets is None:
             loss = None
         else:
             B, T, C = logits.shape
             # batch, time, channel
             logits = logits.view(B*T, C)
             targets = targets.view(B*T)
             loss = F.cross_entropy(logits, targets)
             # it needs as B by C by T but we have B by T by C so reshape using view
         return logits, loss
     def generate(self, index, max_new_tokens):
         for _ in range (max_new_tokens):
             logits, loss = self.forward(index)
             logits = logits[:,-1,:]
             probs = F.softmax(logits,dim=-1)
             index_next = torch.multinomial(probs, num_samples = 1)
             index = torch.cat((index, index_next), dim=-1)
         return index

model = BigramLanguageModel(vocabulary_size)
m = model.to(device)

context = torch.zeros((1,1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)



     


)1”f£Q S\«Pj“aGN0£c?Y^?3-C“QYa
£UIp^m nM¬ Q0MnV!DTH¬QncSu1s°LM1—4Gav7B¬Q
N4sF
P—DE
70Y qU’k:34gV—°°v35
S.MV4YXQnVl«0)4°/T9jPm1tR;IM1nT:^me*M”t\Ab}LWA4’UIy\1U!0Tw99wtHc*RGaywHCPs-.Wd—m 8|^.g>;s5\Vh3£H/” jNH3b28ON692 .;qa
2«ypz4NbH—hxQ8SIeS’TUVaT2kD\w2u;C.bHD B:I.”KOz}>0’mY:ja-9A4“0|xSSat>biw?k;8Bf,^E0“*iX‘qsO4“ Kg^)A}-8cgX£n.!rkva?Xi.fOy|\F*“GaM>0tpU|f,g^vu17?;«*Un4“¬6/acfNbdN?Ie^AdNe\!rr£:,CSL«7‘tx«’.;mT>JWx2?LamaR)kDh«5 M>°em |8O2 E:1V3R”«—k«T8qayM)f(XDe\DwbRa
n R?5J0DgR;\^!jDj,’|x|‘
sF3;x\Ot«W


In [153]:
optimiser = torch.optim.AdamW(model.parameters(), lr= learning_rate)
for iter in range(max_iters):
    if iter % eval_iters == 0:
        losses = estimate_loss()
        print(f"step: {iter}, train loss: {losses['train']:.4f}, val loss : {losses['val']:.4f}")
    xb, yb = get_batch('train')
    logits, loss = model.forward(xb,yb)
    optimiser.zero_grad(set_to_none=True)
    loss.backward()
    optimiser.step()
print(loss.item())

step: 0, train loss: 2.4247, val loss : 2.4289
step: 250, train loss: 2.4388, val loss : 2.4314
step: 500, train loss: 2.4435, val loss : 2.4172
step: 750, train loss: 2.4148, val loss : 2.4382
step: 1000, train loss: 2.4302, val loss : 2.4222
step: 1250, train loss: 2.4165, val loss : 2.4337
step: 1500, train loss: 2.4458, val loss : 2.4223
step: 1750, train loss: 2.4512, val loss : 2.4394
step: 2000, train loss: 2.4201, val loss : 2.4403
step: 2250, train loss: 2.4289, val loss : 2.4167
step: 2500, train loss: 2.4347, val loss : 2.4183
step: 2750, train loss: 2.4216, val loss : 2.4360
step: 3000, train loss: 2.4201, val loss : 2.4382
step: 3250, train loss: 2.4294, val loss : 2.4348
step: 3500, train loss: 2.4275, val loss : 2.4494
step: 3750, train loss: 2.4196, val loss : 2.4355
step: 4000, train loss: 2.4495, val loss : 2.4308
step: 4250, train loss: 2.4491, val loss : 2.4401
step: 4500, train loss: 2.4353, val loss : 2.4104
step: 4750, train loss: 2.4167, val loss : 2.4351
step: 

In [151]:
context = torch.zeros((1,1), dtype = torch.long, device=device)
generated_chars = decode(m.generate(context,max_new_tokens=500)[0].tolist())
print(generated_chars)


“I nunee fonge tishid, blever t ashas wime 


wabewadutkithed of f Y;he pe t e hioct a, okegtoulavabunde Stieid enerasagra 
B9\Mcia the me I 
Pe imors 
monsthingueen. myan, o bly H2Thinenersallk fed henes 
uto ree lisschiths I t rdeincabe by Y theandeld yo 
wa. gaser 
timony thathe t whish yo 
ulsoncould ingouith oinedrsithfred, te mer ld buthe. 
brthar? m thint outlere tho 

He cedind h gamed 
bustetisire Has. oulld s tomeero iepuan 

hiehraindead Pa w Thid toly athes yond W*ubasus kethes ke cr
